<a href="https://colab.research.google.com/github/El-amin/FairCXRnet-A-Multi-Task-Learning-Model-for-Chest-X-Ray-Classification-for-Low-Resource-Settings-/blob/main/MultiTask_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torchvision medmnist


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=4267dc0634927a3616ebf729c20efeddcdc037074ece425813547e314ab26266
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from medmnist import INFO, Evaluator
from medmnist.dataset import ChestMNIST

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
 #1. Load the ChestMNIST dataset
# Define the transformations
transform = transforms.Compose([
    transforms.Resize(224),  # Resize to fit ResNet50 input size
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    #transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize images
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
batch_size = 64

train_dataset = ChestMNIST(split='train', transform=transform, download=True)
val_dataset = ChestMNIST(split='val', transform=transform, download=True)
test_dataset = ChestMNIST(split='test', transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 82802576/82802576 [00:03<00:00, 24817165.07it/s]


Using downloaded and verified file: /root/.medmnist/chestmnist.npz
Using downloaded and verified file: /root/.medmnist/chestmnist.npz


In [ ]:

# Load the DenseNet-201 model
model = models.densenet201(pretrained=True)

# Modify the final fully connected layer for ChestMNIST (14 labels, multi-label classification)
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 14),  # 14 output classes
    nn.Sigmoid()  # Sigmoid for multi-label classification
)

model = model.to(device)

# 3. Define Loss Function, Optimizer, and Scheduler
criterion = nn.BCELoss()  # Binary Cross Entropy for multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:

# 4. Training Function
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.float().to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        scheduler.step()

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch}/{num_epochs}, Loss: {epoch_loss:.4f}')

    return model

# 5. Model Evaluation on Validation Set
def evaluate_model(model, loader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.float().to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

    total_loss = running_loss / len(loader.dataset)
    print(f'Validation Loss: {total_loss:.4f}')



In [ ]:
# 6. Train and Validate the Model
model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)
evaluate_model(model, val_loader)

Epoch 0/10, Loss: 0.1700
Epoch 1/10, Loss: 0.1623


In [ ]:
 #7. Test the Model
def test_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.float().to(device)

            outputs = model(inputs)
            predicted = (outputs > 0.5).float()  # Threshold to make multi-label predictions
            correct += (predicted == labels).sum().item()
            total += labels.numel()

    print(f'Accuracy: {100 * correct / total:.2f}%')

test_model(model, test_loader)